In [2]:
# 텐서플로 임포트 

from __future__ import absolute_import, division, print_function

import tensorflow as tf

tf.enable_eager_execution()

In [ ]:
# 텐서 
    - 텐서 : 다차원 배열 
    - 넘파이의 ndarray 객체와 비슷 / Tensor 객체는 데이터 타입과 크기를 가짐
    - 텐서는 GPU 같은 가속기 메모리에 상주 가능 
    
    ## 넘파이 호환성
        - 텐서플로 연산은 자동적으로 넘파이 배열을 텐서로 변환 -> 텐서는 .numpy() 메서드(method)를 호출하여 넘파이 배열로 변환
            --> 해당 변환이 항상 가능한 것은 아님. (메모리 장소가 GPU이냐 아니냐에 따름)
            --> GPU를 사용중이었다면 GPU에서 호스트 메모리(넘파이 배열이 항상 저장되는 곳)로의 복사가 필요 
        - 넘파이 연산은 자동적으로 텐서를 넘파이 배열로 변환
    

In [3]:
# 텐서 실습 1 

print(tf.add(1, 2))
print(tf.add([1, 2], [3, 4]))
print(tf.square(5))
print(tf.reduce_sum([1, 2, 3]))
print(tf.encode_base64("hello world"))

# 연산자의 오버로딩(overloding) 또한 지원합니다.
print(tf.square(2) + tf.square(3))



tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor([4 6], shape=(2,), dtype=int32)
tf.Tensor(25, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(b'aGVsbG8gd29ybGQ', shape=(), dtype=string)
tf.Tensor(13, shape=(), dtype=int32)


In [4]:
# 텐서 실습 2

# Tensor 객체는 데이터 타입과 크기를 가짐

x = tf.matmul([[1]], [[2, 3]])
print(x.shape)
print(x.dtype)

(1, 2)
<dtype: 'int32'>


In [5]:
# 넘파이 호환성 실습 1

import numpy as np

ndarray = np.ones([3, 3])

print("텐서플로 연산은 자동적으로 넘파이 배열을 텐서로 변환합니다.")
tensor = tf.multiply(ndarray, 42)
print(tensor)


print("그리고 넘파이 연산은 자동적으로 텐서를 넘파이 배열로 변환합니다.")
print(np.add(tensor, 1))

print(".numpy() 메서드는 텐서를 넘파이 배열로 변환합니다.")
print(tensor.numpy())

텐서플로 연산은 자동적으로 넘파이 배열을 텐서로 변환합니다.
tf.Tensor(
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]], shape=(3, 3), dtype=float64)
그리고 넘파이 연산은 자동적으로 텐서를 넘파이 배열로 변환합니다.
[[43. 43. 43.]
 [43. 43. 43.]
 [43. 43. 43.]]
.numpy() 메서드는 텐서를 넘파이 배열로 변환합니다.
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]]


In [ ]:
# GPU 가속기 
    - 대부분의 텐서플로 연산은 GPU를 사용해 가속화 가능 
    - 어떤 주석도 없이 텐서플로는 연산을 위해 자동적으로 CPU 또는 GPU를 사용할 것인지 정함 
    
    ## 장치 이름
        - Tensor.device는 텐서를 구성하고 있는 호스트 장치의 풀네임을 제공
        - 이름은 프로그램이 실행중인 호스트의 네트워크 주소 및 해당 호스트 내의 장치와 같은 많은 세부 정보를 인코딩한다
        - 텐서가 호스트의 N번째 GPU에 놓여지면 문자열은 GPU:<N>으로 끝납니다.
            
    ## 명시적 장치 배치
        - "배치(replacement)"라는 용어는 개별 연산을 실행하기 위해 장치에 할당(배치) 하는 것
        - 언급이 없다면 자동으로 결정 but 위의 장치이름(tf.device)를 사용하여 특정 장치에 명시적으로 배치 가능 
    

In [6]:
# GPU 가속기 실습 1 

x = tf.random_uniform([3, 3])

print("GPU 사용이 가능한가 : "),
## 어떤 주석도 없이 자동적으로 CPU 또는 GPU를 사용할 것인지 정함
print(tf.test.is_gpu_available())

print("텐서가 GPU #0에 있는가 :  "),
print(x.device.endswith('GPU:0'))

GPU 사용이 가능한가 : 
False
텐서가 GPU #0에 있는가 :  
False


In [7]:
# 명시적 장치 배치 실습 1

import time

def time_matmul(x):
  start = time.time()
  for loop in range(10):
    tf.matmul(x, x)

  result = time.time()-start
    
  print("10 loops: {:0.2f}ms".format(1000*result))


# CPU에서 강제실행합니다.
print("On CPU:")
with tf.device("CPU:0"):
  x = tf.random_uniform([1000, 1000])
  assert x.device.endswith("CPU:0")
  time_matmul(x)

# GPU #0가 이용가능시 GPU #0에서 강제실행합니다.
if tf.test.is_gpu_available():
  with tf.device("GPU:0"): # 또는 GPU:1, GPU:2
    x = tf.random_uniform([1000, 1000])
    assert x.device.endswith("GPU:0")
    time_matmul(x)

On CPU:
10 loops: 403.77ms


In [ ]:
# 데이터 셋
    - 모델을 훈련시키고 평가 루프를 제공할 간단하고 재사용 가능한 모듈로부터, 복잡한 입력 파이프라인을 구축하기 위해 데이터셋 API사용 권장 
    - tf.data.Dataset 객체를 통하여 파이썬 반복문을 사용할 수 있으며, 명시적으로 tf.data.Iterator 객체를 생성할 필요가 없습니다. -> 즉시 실행이 활성화될 때에는 반복자에 대한 논의 신경쓰지 않아도 됨.
     
    ## 소스 dataset 생성
        - Dataset.from_tensors, Dataset.from_tensor_slices 와 같은 팩토리 함수 사용 
        - TextLineDataset 또는 TFRecordDataset와 같은 파일로부터 읽어들이는 객체 사용 
        
    ## 변환 적용
        - map, batch, shuffle 과 같은 변환 함수를 사용해 데이터셋의 레코드에 적용하기
        - map : 각 요소별 변형
        - batch : 전체 데이터셋에 대한 변형 
    ## 반복 
        - 즉시 실행이 활성화되면 Dataset 객체는 반복이 가능
        - Dataset.make_one_shot_iterator() 또는 get_next()와 같은 객체를 호출할 필요가 없다
    

In [8]:
# 소스 데이터 셋 생성 실습 1 

ds_tensors = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5, 6])

# CSV 파일을 생성합니다.
import tempfile
_, filename = tempfile.mkstemp()

with open(filename, 'w') as f:
  f.write("""Line 1
Line 2
Line 3
  """)

ds_file = tf.data.TextLineDataset(filename)

In [9]:
# 변환 적용 실습 1 

# map의 함수를 squre를 사용(요소별 제곱해주기)
# shuffle : 버퍼크기 2로 하여 랜덤으로 replace
# batch : 2의 배치 사이즈로 해당 데이터 셋을 combine 

ds_tensors = ds_tensors.map(tf.square).shuffle(2).batch(2)

ds_file = ds_file.batch(2)

In [10]:
# 반복 실습 1 

print('ds_tensors 요소:')
for x in ds_tensors:
  print(x)

print('\nds_file 요소:')
for x in ds_file:
  print(x)

ds_tensors 요소:
Instructions for updating:
Colocations handled automatically by placer.
tf.Tensor([4 1], shape=(2,), dtype=int32)
tf.Tensor([16  9], shape=(2,), dtype=int32)
tf.Tensor([36 25], shape=(2,), dtype=int32)

ds_file 요소:
tf.Tensor([b'Line 1' b'Line 2'], shape=(2,), dtype=string)
tf.Tensor([b'Line 3' b'  '], shape=(2,), dtype=string)
